셀레니움 : 웹테스팅을 위해 만들어진 도구

https://www.selenium.dev/documentation/

* 강점
    1. 브라우저를 직접 실행하는 기능
        - 브라우저로 접근할 수 있는 페이지는 다 접근 가능하다

    2. 브라우저에서 마우스와 키보드 컨트롤 기능
        - 마우스로 클릭할 수 있다
	    - 해당 태그를 파싱할 수 있다
	    - 페이지의 이동이 자유롭다

* 약점
    1. 웹드라이버 프로그램을 설치해야 한다.
    1. 불안정하다.
    1. 느리다.


* tip
    1. selenium에서는 tag를 element라고 명명한다.
    1. xpath를 사용해 element를 찾아서 마우스/키보드 제어
    1. class, id를 사용해 데이터 추출
    1. 찾은 데이터로부터 text_link를 통해 다른 페이지에 접근
    1. 페이지 로드에 시간이 걸릴 수 있으므로 사이에 time delay를 넣어 주어야 한다.
    1. 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)
     


한글 폰트 다운로드 및 설정(선택사항)

In [ ]:
!apt-get update -qq
!apt-get install -y fonts-nanum*
!apt-get install fonts-nanum* -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
The following NEW packages will be installed:
  fonts-nanum fonts-nanum-coding fonts-nanum-eco fonts-nanum-extra
0 upgraded, 4 newly installed, 0 to remove and 59 not upgraded.
Need to get 46.0 MB of archives.
After this operation, 177 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-coding all 2.5-3 [4,988 B]
Get:3 h

## 코랩에서 셀레니움 사용을 위한 환경 세팅

In [ ]:
#https://daydreamx.tistory.com/entry/Colab-Google-Colab%EC%97%90%EC%84%9C-Selenium-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0
!sudo apt -y update
!sudo apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/bin/chromedriver
!pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
59 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

## web driver 생성

크롬 화면을 하나 여는 것과 같다!

In [ ]:
from selenium import webdriver

URL = "https://www.naver.com"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

driver.get(URL)
title = driver.title

print(title)

NAVER


### 1-1. selenium의 기본 사용 코드
#### (1) 엘레멘트 접근 방법
다양한 방법으로 엘레멘트에 접근할 수 있지만 주로 xpath를 사용한다.

주의 : class name에 띄어쓰기가 포함되어 있다면 띄어쓰기 대신 온점('.')으로 꼭 입력해주어야 한다.

(ex. 'sub_time suv_txt' => 'sub_time.suv_text' )


이전버전까지는 find_element 함수가 종류별로 있었으나 find_element, find_elements로 통합되고 argument로 By를 통해 설정해준다.

```
<!-- 과거 버전 -->

driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/button/span[2]')
driver.find_element_by_class_name('ico_search_submit')
driver.find_element_by_id('ke_kbd_btn')
driver.find_element_by_link_text('회원가입')
driver.find_element_by_css_selector('#account > div > a')
driver.find_element_by_name('join')
driver.find_element_by_partial_link_text('가입')
driver.find_element_by_tag_name('input')
driver.get_attribute('href')

```

```
<!-- 현재 버전 -->
driver.find_element(By.ID, '개발자 도구로 찾은 항목')
driver.find_element(By.CLASS, '항목1').find_element(By.CLASS, '항목2')

```

#### (2) 엘레멘트 클릭 / 텍스트 입력 & 삭제
검색창에 텍스트를 입력하고 버튼을 클릭하는 과정을 통해 url 입력 필요없이 다른 페이지로 넘어갈 수 있다.
- 엘레멘트 클릭 => click()
- 텍스트 입력 => send_keys()
- 텍스트 삭제 => clear()

In [ ]:
# 텍스트 입력
driver.find_element(By.ID, 'query').send_keys('코로나')

# 텍스트 삭제
driver.find_element(By.ID, 'query').clear()

# 검색 방법
driver.find_element(By.ID, 'query').send_keys('코로나\n')
# >> 우리가 흔히 검색어를 입력하고 엔터를 치면 검색이 되듯이
#    엔터('\n')를 send_keys()로 입력하면 검색버튼을 누르는 것과 동일하게 된다.

#### (3) 뒤로가기 or 앞으로가기
이전 페이지로 돌아가거나 앞 페이지로 갈 수 있다.

In [ ]:
# 뒤로가기
driver.back()

#앞으로 가기
driver.forward()

#### (4) 스크린샷 찍기
특정 엘레멘트의 스크린샷을 찍어 이미지 파일로 저장이 가능하다.

In [ ]:
#웹페이지 전체 캡쳐
driver.save_screenshot('webpage.png')

#특정 엘레먼트 캡쳐
element = driver.find_element(By.CLASS_NAME, 'news_dsc')
with open('ss.png', 'wb') as f:
    f.write(element.screenshot_as_png)

## 네이버 연관 검색어 찾아보기

In [ ]:
URL = "https://www.naver.com/"
driver.get(URL)


# By.CLASS_NAME
#nx_right_related_keywords > div > div.related_srch > ul > li:nth-child(1) > a > div
driver.find_element(By.ID, 'query').send_keys('코로나\n')
time.sleep(2)
driver.save_screenshot('naver.png')
elements = driver.find_elements(By.CLASS_NAME, 'api_txt_lines')
for e in elements:
    print(e.text)

감염병주요뉴스
김태종 특파원 = 여름철을 맞아 미국에서 신종 코로나바이러스 감염증(코로나19)이 다시 늘어날 조짐을 보이고 있다고 CNN 방송이 28일(현지시간) 보도했다. 미 질병통제예방센터(CDC) 자료에 따르면 최소 38개 주에서 최근 코로나19 감염이 증가하고 있을 가능성이 높은 것으로 나타났다. 지난해 미 공중보건...
황진현 인턴 기자 = 미국에서 신종 코로나바이러스 감염증(코로나19·COVID-19)이 여름을 맞아 다시 늘어날 조짐을 보이고 있는 것으로 전해졌다. 28일(현지시각) CNN은 미 질병통제예방센터(CDC) 자료를 인용해 최소 38개 주에서 최근 코로나19 감염이 늘어날 가능성이 높다고 보도했다. 지난해 미 공중보건...
TS교통환경공단은 코로나19 이후 자동차를 이용한 단체여행객의 수요가 늘고 화물차를 이용한 경제활동이 증가했다고 30일 밝혔다. 사진=데일리한국 안희민 기자 코로나19 이후 경제활동이 회복되자 자동차를 이용한 단체 여행이나 화물운송이 늘어난 것으로 집계됐다. 한국교통안전공단은 작년...
하지만 신종 코로나바이러스 감염증(이하 코로나19) 사태 직후 저금리 시기를 관통하며 쌓인 빚이 그대로인 데다, 고금리 터널의 끝이 아직도 보이지 않는 와중 가계대출이 다시 몸집을 불리고 있다는 점에서 우려가 커질 것으로 보인다. 30일 한국은행에 따르면 은행권의 가계대출 잔액은 지난 달에만...
2019년11월, 중국후베이성우한시에서 처음으로 발생하여 보고된 새로운 유형의 변종 코로나바이러스인 SARS-CoV-2에 의해 발병한 급성 호흡기 전염병이다.
코로나19(신종 코로나바이러스)에 관한 세계 각 국가들의 통계 및 뉴스 등을 취합하여 실시간 정보를 제공합니다.
정의 중국 우한에서 발생한 폐렴은 신종 코로나 바이러스인 ‘2019-nCoV' 가 원인으로 알려졌습니다. 이 신종 바이러스는 2019년 말 처음 인체 감염이 확인됐다는 의미에서 '코로나-19'로 명명되었습니다. 지금까지 코로나 바이러스는 단 여섯 종만이 사람에게 감염되는 것으로

## 네이버 뉴스 가져오기

In [ ]:
URL = 'https://m.search.naver.com/search.naver?ssc=tab.m_news.all&where=m_news&sm=mtb_jum&query=%EC%BD%94%EB%A1%9C%EB%82%98'
driver.get(URL)
titles = driver.find_elements(By.CLASS_NAME, 'api_txt_lines.tit')
for tt in titles:
    print(tt.text)

print('============================')

body = driver.find_elements(By.CLASS_NAME, 'api_txt_lines.dsc_txt')
for b in body:
    print(b.text)

美서 여름철 코로나19 다시 기승 조짐…"38개 주 증가"
미국 38개 주에서 코로나19 확산…"폭염이 확산에 영향"
TS교통안전공단 "코로나19 이후 자동차 단체여행·화물차 이용 증가"
코로나發 빚 아직 쌓여있는데…다시 꿈틀대는 가계대출 '경고등'
온종합병원 “코로나19위기단계 하향조정 후 감염병 급증”
상반기 영화관객 3년 연속 증가, 코로나19 이후 최다 관객
코로나 때보다는 나을라나…'지지부진' 소비 회복[세쓸통]
코로나 경영악화로 대량해고...법원 판단은 "정당"
코로나19 위기단계 하향조정 후 감염병 급증
'두 번째 킬러 배제' 올해 수능 11월14일…'불영어' 불똥 튀나
법원 “항공사가 코로나 경영난으로 조종사 해고한 것 정당해”
지난달 '한국 찾은 외국인' 142만명…미국·대만 코로나 전보다 늘었다
'코로나 등록금 반환' 국립대 학생이 낸 1심도 패소
코로나 끝나자 감염병 환자 급증…작년 코로나 외 감염 17.5%↑
서울 프랜차이즈 가맹점 연매출 평균 3.7억…코로나 이전 보다 많아
김태종 특파원 = 여름철을 맞아 미국에서 신종 코로나바이러스 감염증(코로나19)이 다시 늘어날 조짐을 보이고 있다고 CNN 방송이 28일(현지시간) 보도했다. 미 질병통제예방센터(CDC) 자료에 따르면 최소 38개 주에서 최근 코로나19 감염이 증가하고 있을 가능성이 높은 것으로 나타났다. 지난해...
황진현 인턴 기자 = 미국에서 신종 코로나바이러스 감염증(코로나19·COVID-19)이 여름을 맞아 다시 늘어날 조짐을 보이고 있는 것으로 전해졌다. 28일(현지시각) CNN은 미 질병통제예방센터(CDC) 자료를 인용해 최소 38개 주에서 최근 코로나19 감염이 늘어날 가능성이 높다고 보도했다. 지난해 미...
TS교통환경공단은 코로나19 이후 자동차를 이용한 단체여행객의 수요가 늘고 화물차를 이용한 경제활동이 증가했다고 30일 밝혔다. 사진=데일리한국 안희민 기자 코로나19 이후 경제활동이 회복되자 자동차를 이용한 단체 여행이나 화물운송이 늘어난 것으로 집계됐다...

## 다른 페이지를 선택해서 다른 뉴스 가져오기

In [ ]:
driver.find_element(By.XPATH, '//*[@id="_related_keywords"]/div/div/div/a[5]').click()  # 다른 페이지 하이퍼링크 클릭

## 중간에 delay가 필요할 수 있음
time.sleep(1)
driver.save_screenshot('other_page.png')

titles = driver.find_elements(By.CLASS_NAME, 'api_txt_lines.tit')
for tt in titles:
    print(tt.text)

print('============================')

body = driver.find_elements(By.CLASS_NAME, 'api_txt_lines.dsc_txt')
for b in body:
    print(b.text)

美서 여름철 코로나19 다시 기승 조짐…"38개 주 증가"
미국 38개 주에서 코로나19 확산…"폭염이 확산에 영향"
[단독] 질병청 "수두 백신 문제 아냐" But 논문 '현미경' 분석해보니
SK바이오, 獨 CDMO 인수…“백신 외 사업에서 돌파구 마련”
코로나19 위기단계 하향조정 후 감염병 급증
코로나 백신의 어머니도 ‘아웃사이더’
[임상 업데이트] SK바사, 차세대 폐렴구균 백신 임상 3상 승인
SK바이오사이언스, 글로벌 백신 기업 인수
셀트리온, 백신 개발 의지 재확인…美서 폐렴구균 관련 특허 출원
“백신 없어요” 영·유아 중심 수족구병 한 달 새 급증
온종합병원 “코로나19위기단계 하향조정 후 감염병 급증”
mRNA 전문 모더나, 후속작은 '차세대 코19 백신'과 '독감 백신'
SK바이오사이언스, 독일 백신 위탁생산 기업 인수…"3390억 규모"
SK바사, 獨 백신위탁생산 기업 인수…"글로벌 백신회사 도약"(종합)
SK바이오사이언스, 독일 백신위탁생산 기업 3390억원에 인수
3 및 KP.2 변이가 코로나19 감염의 절반 이상을 차지하고 있다. CDC는 이에 코로나19 백신을 접종한 지 6개월이 넘은 모든 사람에게 업데이트된 코로나19 백신을 접종할 것을 권고했다. 새로운 백신은 8월 중순에서 9월 말 사이에 출시될 것으로 예상된다고 CNN 방송은 전했다.
2에 대응할 약을 제조하라고 권고했다. CNN에 따르면 새로운 백신은 8월 중순에서 9월 말 사이에 출시될 것으로 예상된다. 한편 지난 27일 미 질병통제예방센터는 코로나19 백신을 접종한 지 6개월이 넘은 모든 사람에게 업데이트된 코로나19 백신을 접종할 것을 권고했다．
즉 아이가 과거의 반복적인 감염 병력이 없었더라도 '혹시 수두 백신 접종 이전, 면역이 억제됐거나 저하된 다른 가능성'을 따져보기 위해 보조적인 검사를 추가로 시행했다는 뜻이다. 전문의는 "예를 들어 코로나19 팬데믹 때문에 마스크를 오래 쓰고 위생 관리를 잘하면...
IDT바이오로지카는 전체 매출에서 코로나19 백신 제품이 차

## 네이버 웹툰 가져오기

### 텍스트를 통해 링크 찾기

In [ ]:
URL = 'https://comic.naver.com/webtoon?tab=fri'
driver.get(URL)
time.sleep(3)

driver.find_element(By.PARTIAL_LINK_TEXT, "나 혼자 만렙 뉴비").click()
print(driver.current_url)
driver.save_screenshot('NaHonja.png')
driver.back()  #원래 화면으로 돌아오기

driver.find_element(By.PARTIAL_LINK_TEXT, "외모지상주의").click()
print(driver.current_url)
driver.save_screenshot('goodface.png')
driver.back()

https://comic.naver.com/webtoon/list?titleId=773797&tab=fri
https://comic.naver.com/webtoon/list?titleId=641253&tab=fri


### 전체 웹툰 링크 확보하기

In [ ]:
URL = 'https://comic.naver.com/webtoon?tab=fri'
driver.get(URL)
time.sleep(3)
titles = driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/ul/li')

#웹툰 이름 리스트 추출
name_list = []
for i, t in enumerate(titles):
    if i > 10:
        break

    wt = t.find_elements(By.TAG_NAME, 'span')[1]
    wt_name = wt.text
    print(wt_name)
    name_list.append(wt_name)

print('===================================')
# 이름에 해당하는 링크를 들어가보기
for n in name_list:
    print(n)
    driver.find_element(By.PARTIAL_LINK_TEXT, n).click()
    time.sleep(2)
    print(driver.current_url)
    driver.back()
    time.sleep(2)


외모지상주의
역대급 영지 설계사
좋아? 죽어!
나 혼자 만렙 뉴비
사자의 서
상남자
유부 감자
나 혼자 탑에서 농사
언니, 이번 생엔 내가 왕비야
사신
천하제일 대사형
외모지상주의
https://comic.naver.com/webtoon/list?titleId=641253&tab=fri
역대급 영지 설계사
https://comic.naver.com/webtoon/list?titleId=777767&tab=fri
좋아? 죽어!
https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fcomic.naver.com%2Fwebtoon%2Flist%3FtitleId%3D821192%26tab%3Dfri
나 혼자 만렙 뉴비
https://comic.naver.com/webtoon/list?titleId=773797&tab=fri
사자의 서
https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fcomic.naver.com%2Fwebtoon%2Flist%3FtitleId%3D823737%26tab%3Dfri
상남자
https://comic.naver.com/webtoon/list?titleId=751168&tab=fri
유부 감자
https://comic.naver.com/webtoon/list?titleId=822556&tab=fri
나 혼자 탑에서 농사
https://comic.naver.com/webtoon/list?titleId=808389&tab=fri
언니, 이번 생엔 내가 왕비야
https://comic.naver.com/webtoon/list?titleId=798917&tab=fri
사신
https://comic.naver.com/webtoon/list?titleId=799837&tab=fri
천하제일 대사형
https://comic.naver.com/webtoon/list?titleId=802278&tab=fri


### 웹툰 하나에서 회차별 링크 확보하기

In [ ]:
wt_url='https://comic.naver.com/webtoon/list?titleId=822556&tab=fri'
driver.get(wt_url)
time.sleep(3)

li = driver.find_elements(By.CLASS_NAME, 'EpisodeListList__info_area--Rq03U')
wha_list = []
for l in li:
    wha_list.append(l.text)

for wha in wha_list:
    print(wha)
    driver.find_element(By.PARTIAL_LINK_TEXT, wha).click()
    print(driver.current_url)
    time.sleep(1)
    driver.back()
    time.sleep(1)

34화 햄스터 레레 (1)
별점
8.43
24.06.27
https://comic.naver.com/webtoon/detail?titleId=822556&no=35&week=fri
33화 우당탕탕 결혼식 (3)
별점
9.92
24.06.24
https://comic.naver.com/webtoon/detail?titleId=822556&no=34&week=fri
32화 우당탕탕 결혼식 (2)
별점
9.92
24.06.20
https://comic.naver.com/webtoon/detail?titleId=822556&no=33&week=fri
31화 우당탕탕 결혼식 (1)
별점
9.92
24.06.17
https://comic.naver.com/webtoon/detail?titleId=822556&no=32&week=fri
30화 메리지 블루 (6)
별점
9.80
24.06.13
https://comic.naver.com/webtoon/detail?titleId=822556&no=31&week=fri
29화 메리지 블루 (5)
별점
9.67
24.06.10
https://comic.naver.com/webtoon/detail?titleId=822556&no=30&week=fri
28화 메리지 블루 (4)
별점
9.78
24.06.06
https://comic.naver.com/webtoon/detail?titleId=822556&no=29&week=fri
27화 메리지 블루 (3)
별점
9.72
24.06.03
https://comic.naver.com/webtoon/detail?titleId=822556&no=28&week=fri
26화 메리지 블루 (2)
별점
9.08
24.05.30
https://comic.naver.com/webtoon/detail?titleId=822556&no=27&week=fri
25화 메리지 블루 (1)
별점
9.74
24.05.27


KeyboardInterrupt: 

금요일 모든 웹툰의 모든 회차의 url

### 댓글 추출하기

In [ ]:
wt_url='https://comic.naver.com/webtoon/detail?titleId=822556&no=20&week=fri'
driver.get(wt_url)
time.sleep(3)
#전체 댓글
driver.find_element(By.XPATH, '//*[@id="cbox_module_wai_u_cbox_sort_option_tab2"]/span[2]').click()
time.sleep(0.5)

#더보기 버튼 5번 클릭
for i in range(5):
    driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[7]/a/span/span/span[1]').click()
    time.sleep(0.5)

#댓글 찾기
elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')

review = []
for e in elements:
    review.append(e.text)

print(len(review))
print(review)

89
['스드메 사진 결국 자기가 찍고 싶은 곳애서 하고 싶은 거면 약간 ㅋㅋㅋ!! 퐁이네', '당연한 거긴 한데 당연해지려면 돈을 그만큼 더 줘야지 ㅋㅋㅋ 저작권이 그렇게 중요하다면서 왜 그 저작권 만드는 디자이너를 대우 안해줘서 몰래 경업을 하게 만드냐 이거지', '어쩔수없어요....26살에 결혼하는거면 돈없을때 하는건데 직장인이 더해주는게 맞는거지', '돈없을때지뭐', '가장....은 번듯한 직장있고 거주지 준비해온 남성분인거 같은데', '코로나 이후로 웨딩 시장이 미친듯이 값을 올려서 진짜 차 한 대가 나옵니다^_ㅠ', '진짜 뭐하나 안비싼게 없음 ㅜㅠ', '9살 차이인거 알고나니까 그럴만하네', '아니 근데요 감자님... 저 새벽에 이거 보면서 속상하고 슬프고 눈물나는데 너무 제 처지랑 대입돼서 그런것 같아요...하', '으악....', '나이차가 있으니 당연히 번돈이 많을수 밖에 없고. 또 철밥통이라 해도 정년은 있을테니 작가님이 앞으로 더 벌거라 넘 신경 안쓰셔도 될듯 웹툰 쪽은 정년 없기도 하고', '일단 남편이 여자 보는 눈이 있네 요즘 세상에 저런 여자가 어디있냐 진짜 전설 속 동물 아니냐? 진짜 박스 접기까지 하면서까지 열심히 하는건 대단하네', '이걸로 명작나왔자나요 ㅋㅋ', '당연한거긴 한데...솔직히 돈 더 줄것도 아니면서 저러는거 킹받음', '보기좋구나', '아이고 인스타에서 다봤어서 제가 다 안타깝네여 다 지난일이란 건 알지만..짠내감자ㅠ', 'ㅁㅊ...', '난 직장툰 옛날에 인스타에서 다 봐찌롱 ㅎㅎ 존잼임', '사실 스드메할 돈도 없다면 그냥 소박하게 해도 되는거 아닌가.. 막 겸업까지 하며 속앓이하고까지 할 일들인가 저게', '나도 우리 신랑이 모아놓은돈이 1도 없어서 걍 결혼식 생락함 ㅎㅎ 식당빌려서 밥먹고 집안어르신들 덕담듣고~~', '나이차이가 9살이니 9년치는 남자가 더 벌어서 부담하는게 현실적으로 맞다해도 비슷한 나이로 결혼하는데 한쪽이 모아놓은게 없다? 그건 좀...', 'ㅋㅋㅋㅋ 그림 개웃기네', '슬퍼하지마. 


- 참고:
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)

